# Loading all of the data.

In [55]:
import pandas as pd

regions = ['eune', 'euw', 'na', 'kr', 'jp', 'tr', 'oce', 'br', 'lan', 'ru', 'las']

## Combining all data from the different regions.

In [56]:
df = pd.DataFrame()

for region in regions:
    # Couldn't add the specific region csv files, since it's still collecting data on my pc.
    temp_df = pd.read_csv(f'data_{region}.csv')
    temp_df['region'] = region
    df = df.append(temp_df)

In [57]:
df.head()

,match_id,player1_id,player1_account_id,player1_champion,player1_runes,player2_id,player2_account_id,player2_champion,player2_runes,player3_id,...,player9_id,player9_account_id,player9_champion,player9_runes,player10_id,player10_account_id,player10_champion,player10_runes,won,region
0,2244712072,1FPXTKJY3a3_xqZSA2PuR4t_Ysf6cwBKg_Q4tQYOriqrgk0,tib1-9bPbNYxTX3hunMxa_F-eMX6U3yjbVCIWfO-aL0vXrA,120,Conqueror,jejXtucxhfZe-t8nP-j5mFI7u3Cg5ImCk_EBLr5I0GCftJg,dXYrb_vB1rKSgaAqVX_S1Wo-Tbxpq9K4rhG4GEH9jGUJZw,99,Arcane Comet,aNokvrQ_EDxBtjio7dV3bpkCpLuUlaAq-qpN61ud0M7dBHY,...,47XDcOwZBcoDyaXtU1FGXqbiiXvKd6iYsrSRWRB4tNT1cEc,QC2IgRofk5dv6CeRfjh7LlaRTnkRsWubxdGztjoLTuneTg8,516,Aftershock,YCW3vmVcIDCvnO55ugsEO2EggjYNhG5FhJyzG_0NVwGCcMs,2yy9k5Lw8BR1MeLJiWJfroqSabrK6O1fnRIiQ9AhFxoahKE,145,Press the Attack,False,eune
1,2240827183,xGEOzj2sgTy4CEuq-syGWzju6ClkOANKYj_NIXu1r-zEkTg,IYb_FLgpjG8Sr5P8ASnMD1anyh7S1dCvvj2LlniNO2E_ADU,6,Conqueror,fnbkuFolmbsJ6z0vuG_Csvn-Bnm5qAEJQ_6RyIXj0SMKqgo,0aDABRqXhHKLBOhMZ0x6cPD_Z2g0Mzbe6zm_SBfdt63QEw,429,Lethal Tempo,Tr8eFivfvWZaULv-n3-d1NgGt0eXIVsDWFWzymk5PCNH2FU,...,QvKYTdLPYFQmpCcD4p9wKgKxsGDuTgVLRdy6ov9JBUB0-G0,7fXgIAkjRSQfCDY46ozX8Tb7n_ZopUfycHRuu1sLYKUFAA,15,Lethal Tempo,9QkES98_bU_HMAG6E2KLW0D_9Z4X7W7_GjTpjVvSvohq3AU,fvsuJhUCpkJm8VWn_zkPz9kBkSHuiq2Ue961ZI2adX4K9sM,114,Grasp of the Undying,True,eune
2,2240834456,7PLXLOBh8HFMLvJxTPZHxhcwSvnXd1TTISZaxyl2dNshcds,-q6PxquIlnc3fP0_unBWttYmtJEJa4jJISMzduUE2mo2sw,164,Grasp of the Undying,9jlyDW-3i1K7W1AedSyLUQdhe3ufY3IN1mp-dnsagpmXwpA,1lNmC_horeujp4HNQrmyNLRHQq2XP45O9SDV2AtoxpUH6g,51,Press the Attack,SUuc-eukgIdMD4PXsPo0QEHRdiAv8EbdF5axOpm-ibOiPgs,...,ijHifl95c7SzBSwZoDgYHPt6H0mFP72xuahir5vM0hoP2BY,E1EZ5Jd8mjcoYrbjpizWUPmYmyUZWHsXisoRdGe9R-iGrZk,39,Conqueror,antauAOO-jfd6KVfYW6Sdj-BUZzUoKj2jp4MS5rVLWJTdSs,dPXLFX_9eFW5kSS8sFB2GffqDYfP_cDOtJPyFTIsSF9lcw,236,Press the Attack,True,eune
3,2242271553,1KWmAoQ8RwjRj6QtuO0KnrlNMwaWI7RdCeu-ZO7ANomZVqo,CHm_krX8O8AD9ZmEuODrQmGTrvPB-KzMhijgsw71GokZgA,42,Fleet Footwork,h8vlBYgBYDZDssGBFAytVbM4CghSVxJ6g9sPvSEL8OuvBTc,Gycw0Jxa0Ct1CE8WTt49K4xGPneQMhE67Snv9cNs8ihQhrs,59,Conqueror,hGuAwF7t6tmPiPBAmPKUhriYMOwujBBh0WINM5xvUgarJ3o,...,IgwtJDWO4BVGlV-w9u454xiZcTZ5pTe0Uc-7CesxpXeswHs,FyBcfMTg9z6kgPUscPWpmWJEvSaLQWkwNuXY9gtua1DhJf...,28,Electrocute,ijHifl95c7SzBSwZoDgYHPt6H0mFP72xuahir5vM0hoP2BY,E1EZ5Jd8mjcoYrbjpizWUPmYmyUZWHsXisoRdGe9R-iGrZk,81,Kleptomancy,False,eune
4,2240907415,VwgAEtb3PzycTOwxHdAvj9YZ4Q_aDWAiHxsoLCawGv6Wrpk,PvTAlzlO4Nmq88p3CEQQ9Qq6ki7YFrGzGy35tJQmuopnm-w,84,Electrocute,6BHpYRmwIroV4EqRItNKTPXWYjk32f7eOE0JefurUCq0Wec,CDXHBvtZa63i6cObFWIRzlrcjWAzK2E42tAg3ISOJssMk-4,60,Electrocute,zYBoBWJ8jlAxjCm7P4T1E-dESUHkYTIFsn2JK94zR9H4NvU,...,IJvIAJuutxOUS_WK6bOWGD2kn1IzwJUtlchE5Z56QKQi5ck,QQ2utqo-RDYtKqJG0R-eFNRpxSNu5CUk4O22lDkB8R_GIBs,24,Conqueror,8YgxUXg1hRvmKaoNUqtUS556Ub1XJUTYi-hmno_gcrzvqt4,p-Iv-6aLEFMlozgJRoUtMy26OqaqNxCazRkqygdsQuF2O9Q,517,Conqueror,True,eune


In [58]:
# The number of rows is low because of a mistake I didn't notice when collecting data,
# so I had to work on getting more data again.
# I'll also be using 3rd party websites to scrape for more data.
# I already have a plan on what I'm looking for, champion win rates, 
# champion play rate, and average damage dealt for each picked champion.

df.shape

(110786, 43)

In [59]:
# Save all of the data into one csv file for easier managing later on.
# Plus 'data.csv' is too large to upload to github enterprise, so uploading all files seperately is better.

df.to_csv('data.csv', index=None, header=True)

I have a few other features that I think would work, but I don't know how to apply correctly.
I think adding previous games of the player on a certain champion, or the current role would help with the prediction, with older matches having less weight than recent ones.

I can do this easily since I can query the api using timestamps that can be easily gotten from the match id.